In [397]:
import numpy as np
import cv2
import colorsys

In [398]:
colors = []
for h in range(0, 360, 40): # H
    for s in range(0, 101, 20): # S
        for v in range(0, 101, 20): # V
            r, g, b = colorsys.hsv_to_rgb(h / 360, s / 100, v / 100)
            colors.append([int(r * 255), int(g * 255), int(b * 255)])
palette = np.array(colors)

def quantize_to_palette(image):
    X_query = image.reshape(-1, 3).astype(np.float32)
    X_index = palette.astype(np.float32)

    knn = cv2.ml.KNearest_create()
    knn.train(X_index, cv2.ml.ROW_SAMPLE, np.arange(len(palette)))
    ret, results, neighbours, dist = knn.findNearest(X_query, 1)

    quantized_image = np.array([palette[idx] for idx in neighbours.astype(int)])
    quantized_image = quantized_image.reshape(image.shape)
    return quantized_image


def extract_ring_2(path, index):
    og_img = cv2.imread(path)
    B, G, R = cv2.split(og_img)
    B_norm = cv2.normalize(B, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    G_norm = cv2.normalize(G, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    R_norm = cv2.normalize(R, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    normalized_image = cv2.merge([B_norm, G_norm, R_norm])
    img = np.uint8(normalized_image)

    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    s = s.astype(np.float32)
    s = s * 1.5
    v = v * 1.5
    s = np.clip(s, 0, 255).astype(np.uint8)
    v = np.clip(v, 0, 255).astype(np.uint8)
    hsv_modified = cv2.merge([h, s, v])
    img = cv2.cvtColor(hsv_modified, cv2.COLOR_HSV2BGR)

    dimensions = (360, 640)
    scale_factor_x = og_img.shape[1] / dimensions[0]
    scale_factor_y = og_img.shape[0] / dimensions[1]
    img = cv2.resize(img, (dimensions), interpolation=cv2.INTER_AREA)

    # refine once
    lower2 = np.array([50, 0, 100])     # darken
    upper2 = np.array([255, 150, 255])
    mask2 = cv2.inRange(img, lower2, upper2)
    img = cv2.bitwise_and(img, img, mask=mask2)
    B, G, R = cv2.split(img)
    B_norm = cv2.normalize(B, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    G_norm = cv2.normalize(G, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    R_norm = cv2.normalize(R, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    normalized_image = cv2.merge([B_norm, G_norm, R_norm])
    img = np.uint8(normalized_image)
    # cv2.imwrite(f'temp.png', img)

    # actual thresholding time
    lower = np.array([100, 0, 150])     # darken
    upper = np.array([255, 200, 255])
    mask = cv2.inRange(img, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (4, 4))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    # white_mask = cv2.inRange(img, (180, 180, 180), (255, 255, 255))
    # mask = cv2.subtract(mask, white_mask)
    segmented = cv2.bitwise_and(img, img, mask=mask)
    # cv2.imwrite(f'temp2.png', segmented)

    # Find contours
    gray = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(segmented, [largest_contour], -1, (0, 255, 0), 2)

    upscaled_contour = largest_contour.copy().astype(np.float32)
    upscaled_contour[:, :, 0] *= scale_factor_x
    upscaled_contour[:, :, 1] *= scale_factor_y
    upscaled_contour = upscaled_contour.astype(np.int32)

    # Create a blank mask with the same dimensions as the image
    mask_2 = np.zeros_like(og_img)
    cv2.drawContours(mask_2, [upscaled_contour], -1, (255, 255, 255), thickness=cv2.FILLED)
    white_background = np.full_like(og_img, 255)
    masked_image = np.where(mask_2 == 255, og_img, white_background)

    cv2.imwrite(f'test1.png', masked_image)

extract_ring_2('img_1.jpg', 0)